# 39. Decorators || Decoradores
function decorators:
- call functions-arguments
- modify function behavior

decoradores de funciones:
- llamar a funciones-argumentos
- modificar el comportamiento de la función

In [26]:
# functions without arguments / funciones sin argumentos
def simple_decorator(func):
    def wrapper():
        print("before the function call")
        func()
        print("after the function call")
    return wrapper
@simple_decorator
def say_hello(n=3):
    print(n * "hello! hola! ")
say_hello, say_hello()

before the function call
hello! hola! hello! hola! hello! hola! 
after the function call


(<function __main__.simple_decorator.<locals>.wrapper()>, None)

In [27]:
from datetime import datetime
def during_business_hours(func, start=8, end=20):
    def wrapper():
        if start <= datetime.now().hour < end: func()
        else: pass
    return wrapper
# it works only in the certain time period (8-20)
# funciona sólo en un período de tiempo determinado (8-20)
@during_business_hours
def say_hi():
    print("hi!")
say_hi()

hi!


In [28]:
# functions with arguments and return values
# funciones con argumentos y valores de retorno
def print_return(func):
    def wrapper_twice(*args, **kwargs):
        print(func(*args, **kwargs))
        return func(*args, **kwargs)
    return wrapper_twice
@print_return
def upperlower_string(string):
    return string.upper(), string.lower()
u, l = upperlower_string("HeLlO")
u

('HELLO', 'hello')


'HELLO'

In [29]:
# an universal decorator / un decorador universal
def upper_deco(f):
    def inf(*arg, **kwarg):
        return f(*arg, **kwarg).upper()
    return inf
@upper_deco
def hello(name):
    return f'hello, {name}'
@upper_deco
def sum_xy(x, y):
    return f'{x = }, {y = } => {x + y = }'
print(hello('tim'), sum_xy(2, 3), sep='\n')

HELLO, TIM
X = 2, Y = 3 => X + Y = 5


In [30]:
# several decorators / varios decoradores
def split_deco(f):
    def inner(*arg, **kwarg):
        return f(*arg, **kwarg).split()
    return inner
@split_deco
@upper_deco
def hi(n):
    return n * 'hi '
hi(5)

['HI', 'HI', 'HI', 'HI', 'HI']

In [31]:
# information about the arguments with call count
# información sobre los argumentos con el recuento de llamadas
import functools, math
def print_args(f, c=5):
    @functools.wraps(f)
    def inner(*args, **kwargs):
        inner.num_calls += 1
        value = f(*args, **kwargs)
        if inner.num_calls <= c:
            print(f'{f.__name__}({args=}, {kwargs=}) = {value}')
        return value
    inner.num_calls = 0
    return inner

@print_args
def dot_product(a=(1, 2), b=(2, 1)):
    return sum(a[i] * b[i] for i in range(len(a)))
print(dot_product((2, 3, 4, 5), (6, 7, 8, 9)),
      dot_product((20, 30, 40), b=(50, 60, 70)))

math.factorial = print_args(math.factorial)
def calculate_e(n=10):
    return sum(1 / math.factorial(n) for n in range(n))
print(calculate_e(5))

dot_product(args=((2, 3, 4, 5), (6, 7, 8, 9)), kwargs={}) = 110
dot_product(args=((20, 30, 40),), kwargs={'b': (50, 60, 70)}) = 5600
110 5600
factorial(args=(0,), kwargs={}) = 1
factorial(args=(1,), kwargs={}) = 1
factorial(args=(2,), kwargs={}) = 2
factorial(args=(3,), kwargs={}) = 6
factorial(args=(4,), kwargs={}) = 24
2.708333333333333


In [32]:
# information about a running time
# información sobre un tiempo de ejecución
import time
inner_functions = dict()
def timer1(f, inner_functions=inner_functions):
    def wtimer(*args, **kwargs):
        start = time.perf_counter()
        value = f(*args, **kwargs)
        end = time.perf_counter()
        string = f"function: {f.__name__!r}; "
        string += f"run time: {(end - start):.4f} seconds"
        print(string)
        inner_functions[f.__name__] = f
        return value
    return wtimer
@timer1
def n2sum(n):
    return sum([sum([i**2 for i in range(n)]) for _ in range(n)])
[inner_functions, n2sum], help(n2sum), n2sum(5 * 10 ** 3)

Help on function wtimer in module __main__:

wtimer(*args, **kwargs)

function: 'n2sum'; run time: 8.8471 seconds


([{'n2sum': <function __main__.n2sum(n)>},
  <function __main__.timer1.<locals>.wtimer(*args, **kwargs)>],
 None,
 208270837500000)

In [33]:
import functools, time
def timer2(f):
    @functools.wraps(f)
    def wtimer(*args, **kwargs):
        start = time.perf_counter()
        value = f(*args, **kwargs)
        end = time.perf_counter()
        string = f"function: {f.__name__!r}; "
        string += f"run time: {(end - start):.4f} seconds"
        print(string)
        return value
    return wtimer
@timer2
def n3sum(n):
    return sum([sum([i**3 for i in range(n)]) for _ in range(n)])
n3sum, help(n3sum), n3sum(5 * 10 ** 3)

Help on function n3sum in module __main__:

n3sum(n)

function: 'n3sum'; run time: 10.3007 seconds


(<function __main__.n3sum(n)>, None, 780937531250000000)

In [34]:
import time
def cycle(lst, start=None):
    start = 0 if (start is None) else lst.index(start)
    while True:
        yield lst[start]
        start = (start + 1) % len(lst)
colors1, colors2 = [20, 34, 164, 196], [40, 129, 200, 202]
def colored(func, colors=colors1, end='\n'):
    def wrapper(*args, **kwargs):
        args = (f'\033[1;38;5;{c}m{a}' for (c,a) in zip(cycle(colors), args))
        kwargs['end'] = end
        return func(*args, **kwargs)
    return wrapper
def cprint(*args, **kwargs):
    for a in args:
        print(a, **kwargs, flush=True)
        time.sleep(1)
    print()
cprint1 = colored(cprint)
cprint1(*range(4))
cprint2 = colored(cprint, colors=colors2, end=' ')
cprint2(*range(20))

0
1
2
3

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 


In [35]:
# @ decorators with arguments / @ decoradores con argumentos
import functools
def repeat(times):
    def deco_repeat(f):
        @functools.wraps(f)
        def inner_repeat(*args, **kwargs):
            for _ in range(times):
                value = f(*args, **kwargs)
            return value
        return inner_repeat
    return deco_repeat
@repeat(5)
def print_emo(emo='📓'):
    print(emo, end=' ')
    return emo
emo = print_emo()

📓 📓 📓 📓 📓 

In [36]:
def repeat2(_f=None, *, times=2):
    def deco_repeat(f):
        @functools.wraps(f)
        def wrapper_repeat(*args, **kwargs):
            value = None
            for _ in range(times):
                value = f(*args, **kwargs)
            return value
        return wrapper_repeat
    if _f:
        return deco_repeat(_f)
    else:
        return deco_repeat
@repeat2
def print2emo(emo='📓'):
    print(emo, end=' ')
    return emo
emo = print2emo()
@repeat2(times=10)
def printNemo(emo='📓'):
    print(emo, end=' ')
    return emo
emo = printNemo()

📓 📓 📓 📓 📓 📓 📓 📓 📓 📓 📓 📓 

In [37]:
import functools, time
@functools.lru_cache(maxsize=None)
def fibonacci1(n):
    if n < 2: return n
    return fibonacci1(n-1) + fibonacci1(n-2)
begin1 = time.time()
fibonacci1(20)
end1 = time.time()
print(f'{end1 - begin1: .8f}')

def fibonacci2(n):
    if n < 2: return n
    return fibonacci2(n-1) + fibonacci2(n-2)
begin2 = time.time()
fibonacci2(20)
end2 = time.time()
print(f'{end2 - begin2: .8f}')

 0.00013566
 0.00453544


In [38]:
import functools, time
# a cache of previous function calls
# un caché de llamadas anteriores a la funcion
def cache(func):
    @functools.wraps(func)
    def wrapper_cache(*args, **kwargs):
        cache_key = args + tuple(kwargs.items())
        if cache_key not in wrapper_cache.cache:
            wrapper_cache.cache[cache_key] = func(*args, **kwargs)
        return wrapper_cache.cache[cache_key]
    wrapper_cache.cache = dict()
    return wrapper_cache
@cache
def fibonacci3(n):
    if n < 2: return n
    return fibonacci3(n-1) + fibonacci3(n-2)
fibonacci3(20)
begin3 = time.time()
fibonacci3(10)
end3 = time.time()
print(f'{end3 - begin3: .8f}')

 0.00007463


In [39]:
!pip install -q pint

In [40]:
def set_unit(unit):
    def deco_unit(func):
        func.unit = unit
        return func
    return deco_unit
@set_unit("cm^3")
def cylinder_volume(radius, height, precision=8, pi=3.14159265):
    return round(pi * radius**2 * height, precision)
print(cylinder_volume(2, 3), cylinder_volume.unit)

37.6991118 cm^3


In [41]:
import pint
unit_reg = pint.UnitRegistry()
v = cylinder_volume(2, 3) * unit_reg(cylinder_volume.unit)
print(v, v.to("cubic meters"), v.to("cubic inches"), sep=', ')

37.6991118 centimeter ** 3, 3.76991118e-05 meter ** 3, 2.300540951081903 inch ** 3


In [42]:
# different behaviors depending upon the type of its first argument
# diferentes comportamientos dependiendo del tipo de su primer argumento
from functools import singledispatch
@singledispatch
def example(s): return s
@example.register(int)
def _(s): return s ** 2
@example.register(float)
def _(s): return s * 2
print(example('hello'), example(5), example(.5))

hello 25 1.0


Decorators in Classes || Decoradores en clases

In [43]:
class Celsius:
    def __init__(self, temperature=0):
        self.temperature = temperature
    @property
    def fahrenheit(self):
        return (self.temperature * 1.8) + 32
c = Celsius(24)
print(c.temperature, c.fahrenheit)

24 75.2


In [44]:
class Point:
    def __init__(self):
        self.__x = 0
        self.__y = 0
    @property
    def x(self):
        return self.__x
    @property
    def y(self):
        return self.__y
    @x.setter
    def x(self, value):
        self.__x = value
    @y.setter
    def y(self, value):
        self.__y = value
    def zero_distance(self):
        return (self.__x ** 2 + self.__y ** 2) ** .5
p = Point()
print(p.x, p.y)
p.x, p.y = 30, 40
print(p.x, p.y, p.zero_distance())

0 0
30 40 50.0


In [45]:
class Math:
    @staticmethod
    def vector_sum(x, y):
        if len(x) != len(y):
            print("\033[1;38;5;202mWarning: \033[0m"
                  "vectors must have the same dimensions for summation.")
            return None
        else:
            return [x[i] + y[i] for i in range(len(x))]
m = Math()
Math.vector_sum([1, 2, 3], [4, 5]), m.vector_sum([1, 2], [4, 5])

(None, [5, 7])

In [46]:
class Circle:
    pi = 3.14159265
    def __init__(self, radius, unit=None):
        self.radius = radius
        self.unit = unit
    def __str__(self):
        if self.unit:
            return f"Circle(radius={self.radius}, unit={self.unit})"
        else:
            return f"Circle(radius={self.radius})"
    @classmethod
    def from_diameter(cls, diameter, unit=None):
        radius = diameter / 2
        return cls(radius, unit)
    @property
    def area(self):
        value = round(self.pi * self.radius ** 2, 8)
        if self.unit:
            return value, self.unit + '^2'
        else:
            return value
    @property
    def circumference(self):
        value = round(2 * self.pi * self.radius, 8)
        if self.unit:
            return value, self.unit
        else:
            return value
c1 = Circle.from_diameter(10)
print(c1, c1.area, c1.circumference)
c2 = Circle.from_diameter(10, 'cm')
print(c2, c2.area, c2.circumference)

Circle(radius=5.0) 78.53981625 31.4159265
Circle(radius=5.0, unit=cm) (78.53981625, 'cm^2') (31.4159265, 'cm')


In [47]:
class GenericTypeClass():
    def __init__(self, d={}):
        self.type = None
        for key, value in d.items():
            setattr(self, key, value)
    def __str__(self):
        d = self.__dict__
        g = (f"{k}={d[k]}" for k in d.keys() if k !='type')
        return f"{d['type']}({', '.join(g)})"
    @classmethod
    def from_tuple_list(cls, list):
        return cls(dict(list))
    @property
    def keys(self):
        return list(f"{k}" for k in self.__dict__.keys() if k !='type')
point_dict1 = {'type':'Point2D', 'id':1, 'x':10, 'y':20}
p1 = GenericTypeClass(point_dict1)
print(p1, p1.keys)
point_lst2 = [('type', 'Point3D'), ('id', 2), ('x', 10), ('y', 20), ('z', 30)]
p2 = GenericTypeClass.from_tuple_list(point_lst2)
print(p2, p2.keys)
t = GenericTypeClass()
print(t.type, t.keys)

Point2D(id=1, x=10, y=20) ['id', 'x', 'y']
Point3D(id=2, x=10, y=20, z=30) ['id', 'x', 'y', 'z']
None []


In [48]:
from functools import total_ordering
# fills in the missing comparison methods
# completa los métodos de comparación que faltan
@total_ordering
class Emo:
    def __init__(self, value, emo):
        self.value = value
        self.emo = emo
    def __str__(self):
        return self.value * self.emo
    def __eq__(self, other):
        return (self.value == other.value) and (self.emo == other.emo)
    def __lt__(self, other):
        return (self.value < other.value) and (self.emo == other.emo)
lemon31, apple3 = Emo(3, '🍋'), Emo(3, '🍎')
lemon32, apple5 = Emo(3, '🍋'), Emo(5, '🍎')
print(lemon31, apple3, lemon32, apple5)
print(lemon31 == apple3, lemon31 != apple3, lemon31 < apple3, lemon31 == lemon32,
      apple3 == apple5, apple3 != apple5, apple3 > apple5, apple3 < apple5)

🍋🍋🍋 🍎🍎🍎 🍋🍋🍋 🍎🍎🍎🍎🍎
False True False True False True False True


In [49]:
from dataclasses import dataclass
@dataclass
class Point3D:
    x: float
    y: float
    z: float
    def __str__(self):
        x, y, z = self.x, self.y, self.z
        return f"Point3D({x=:f}, {y=:f}, {z=:f})"
p3d = Point3D(10.1, 5.555555555555, 0)
print(p3d, p3d.__annotations__, sep='\n')

Point3D(x=10.100000, y=5.555556, z=0.000000)
{'x': <class 'float'>, 'y': <class 'float'>, 'z': <class 'float'>}


Classes as decorators || Clases como decoradores

In [54]:
class SimpleDecorator:
    def __init__(self, f):
        self.f = f
    def __call__(self, *args, **kwargs):
        print(f"before the call of {self.f.__name__!r}")
        result = self.f(*args, **kwargs)
        print(f"after the call of {self.f.__name__!r}")
        return result
@SimpleDecorator
def my_function():
    print("'my_function' is called")
my_function()

before the call of 'my_function'
'my_function' is called
after the call of 'my_function'


In [51]:
class OuterDecorator:
    def __init__(self, symbol, num):
        self.symbol = symbol
        self.num = num
    def __call__(self, func):
        def wrapper(*args, **kwargs):
            print(self.num * f"{self.symbol}")
            result = func(*args, **kwargs)
            print(self.num * f"{self.symbol}")
            return result
        return wrapper
class InnerDecorator:
    def __init__(self, symbol, num):
        self.symbol = symbol
        self.num = num
    def __call__(self, func):
        def wrapper(*args, **kwargs):
            print("\t" + self.num * f"{self.symbol}")
            result = func(*args, **kwargs)
            print("\t" + self.num * f"{self.symbol}")
            return result
        return wrapper
@OuterDecorator("=outer=", 5)
@InnerDecorator("=inner=", 3)
def my_function2():
    print("inside 'my_function2'")
my_function2()

=outer==outer==outer==outer==outer=
	=inner==inner==inner=
inside 'my_function2'
	=inner==inner==inner=
=outer==outer==outer==outer==outer=


In [88]:
import functools
# classes as decorators with properties, messages, and arguments
# clases como decoradores con propiedades, mensajes y argumentos
class ComplexPropertyDecorator:
    def __init__(self, message_before, message_after):
        self.message_before = message_before
        self.message_after = message_after
    def __call__(self, func):
        @functools.wraps(func)
        def wrapper(obj):
            print(f"{self.message_before} the property access")
            result = func(obj)
            print(f"{self.message_after} the property access")
            return result
        return wrapper
class DecoratedPropertyClass:
    @property
    @ComplexPropertyDecorator( "before", "after")
    def deco_property(self):
        return "property values"
dpc = DecoratedPropertyClass()
print(dpc.deco_property)

before the property access
after the property access
property values


In [84]:
# classes as decorators with methods, messages, and arguments
# clases como decoradores con métodos, mensajes y argumentos
import functools
class ComplexMethodDecorator:
    def __init__(self, method, message_before, message_after):
        functools.update_wrapper(self, method)
        self.method = method
        self.message_before = message_before
        self.message_after = message_after
    def __get__(self, instance, owner):
        return type(self)(self.method.__get__(instance, owner),
                          self.message_before,
                          self.message_after)
    def __call__(self, *args, **kwargs):
        print(f"{self.message_before} the method call")
        result = self.method(*args, **kwargs)
        print(f"{self.message_after} the method call")
        return result
class DecoratedMethodClass:
    def deco_method(self, value):
        return value
    deco_method = ComplexMethodDecorator(deco_method, "before", "after")
dmc = DecoratedMethodClass()
print(dmc.deco_method('hello'))

before the method call
after the method call
hello


In [52]:
import functools
# the decorator for call counting
# el decorador para el conteo de llamadas
class NumCalls:
    def __init__(self, f):
        functools.update_wrapper(self, f)
        self.f = f
        self.ncalls = 0
    def __call__(self, *args, **kwargs):
        self.ncalls += 1
        num_calls = self.ncalls
        print(f'{self.f.__name__!r}({args=}, {kwargs=}): {num_calls=}')
        return self.f(*args, **kwargs)
@NumCalls
def recursive_ternary_sum(n):
    return n + recursive_ternary_sum(n - 1) if n > 0 else 0
print(recursive_ternary_sum(3))
rts2 = recursive_ternary_sum(3)

'recursive_ternary_sum'(args=(3,), kwargs={}): num_calls=1
'recursive_ternary_sum'(args=(2,), kwargs={}): num_calls=2
'recursive_ternary_sum'(args=(1,), kwargs={}): num_calls=3
'recursive_ternary_sum'(args=(0,), kwargs={}): num_calls=4
6
'recursive_ternary_sum'(args=(3,), kwargs={}): num_calls=5
'recursive_ternary_sum'(args=(2,), kwargs={}): num_calls=6
'recursive_ternary_sum'(args=(1,), kwargs={}): num_calls=7
'recursive_ternary_sum'(args=(0,), kwargs={}): num_calls=8
